In [305]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from utils_accelerate import *
import sentencepiece as spm

# tokenizer = T5Tokenizer.from_pretrained('t5-small')
dataset_name = 'wikidata5m'
# input = "predict tail: barack obama | position_held |"small)1
# input = "translate English to German: How are you doing?"

# model = T5ForConditionalGeneration.from_pretrained('models/codex_m_accelerate_1gpu.pt')
# checkpoint_location = 'models/codex_m_accelerate_1gpu/115000.pt'

# checkpoint_location = 'models/codex-s_small_1gpu/20000.pt'
# checkpoint_location = 'models/codex-m_tiny/70000.pt'
# checkpoint_location = 'models/codex-m_small_6gpu/45000.pt'
checkpoint_location = 'models/wikikg90mv2_small/920000.pt'
checkpoint_location = 'models/wq_small/5500.pt'
model = load_accelerator_model(checkpoint_location, only_model=True)
model.eval()
# model.cpu()
# model.cuda()
model.device

Model args
Namespace(batch_size=80, dataset='web_questions', epochs=50, force_lr=1, hf_dataset=1, hops=1, large_dataset=0, learning_rate=0.0005, load_checkpoint='wikikg90mv2_small/1020000.pt', loss_steps=250, max_checkpoints=5, max_input_sequence_length=40, max_output_sequence_length=30, model_size='t5-small', num_workers=4, optimizer='adafactor', pad_to_max=False, relation_prediction=0, resume=None, save_prefix='wq_small', save_steps=500, start_steps=0, task='qa', tokenizer='t5', use_lm_pretraining=0)


 22%|████████████▎                                            | 81471/376358 [00:00<00:00, 814704.95it/s]

Vocab size is 32100


100%|████████████████████████████████████████████████████████| 376358/376358 [00:00<00:00, 901225.68it/s]


device(type='cpu')

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
tokenizer.vocab_size

32100

In [205]:
location = '/scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2'
model.save_pretrained(location)

In [11]:
tokenizer.save_pretrained(location)

('/scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2/tokenizer_config.json',
 '/scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2/special_tokens_map.json',
 '/scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2/spiece.model',
 '/scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2/added_tokens.json')

In [226]:
# save in tensorflow format
from transformers import TFT5ForConditionalGeneration
location = '/scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2'
tf_model = TFT5ForConditionalGeneration.from_pretrained(location, from_pt=True)

2022-02-11 18:23:40.395501: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-11 18:23:40.434086: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-11 18:23:40.796449: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-02-11 18:23:40.797731: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-02-11 18:23:40.798989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:06:0

In [229]:
tf_model.save_pretrained(location)

In [225]:
from transformers import AutoModel

model2 = AutoModel.from_pretrained(location)

Some weights of the model checkpoint at /scratche/home/apoorv/hf_model_repos/kgt5-wikikg90mv2 were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
# from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
# config = T5Config().from_pretrained('t5-small')

In [14]:
# config

In [51]:
def greedyPredict(input, model, tokenizer):
    input_ids = tokenizer([input], return_tensors="pt").input_ids
    out_tokens = model.generate(input_ids)
    out_str = tokenizer.batch_decode(out_tokens, skip_special_tokens=True)
    return out_str[0]

def greedyPredictBatch(input, model, tokenizer):
    tokenized = tokenizer(input, return_tensors="pt")
    out_tokens = model.generate(**tokenized)
    out_str = tokenizer.batch_decode(out_tokens, skip_special_tokens=True)
    return out_str

In [243]:
subject = "T. C. Achintya"
relation = "family name"
input = subject + "| " + relation
# input = "Queen Victoria| related to"
# input = "Tokyo Olympics 2016| host country"
out = greedyPredict(input, model, tokenizer)
out

'Achintya'

In [242]:
input = "predict answer: what is the name of justin bieber brother?"
out = greedyPredict(input, model, tokenizer)
out

'Jazmyn Bieber'

In [55]:
greedyPredictBatch(["Q42342344| follows"], model, tokenizer)

['Q4234279']

In [64]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("apoorvumang/kgt5-wikikg90mv2")

model = AutoModelForSeq2SeqLM.from_pretrained("apoorvumang/kgt5-wikikg90mv2")

In [230]:
# try datasets
from datasets import load_dataset

dataset = load_dataset("web_questions")

Using custom data configuration default


Dataset web_questions downloaded and prepared to /home/apoorv/.cache/huggingface/datasets/web_questions/default/1.0.0/e6742cc64f6652db0c52cb07b5414e3c001512bf5bde7aa5587353c31db1ed8b. Subsequent calls will reuse this data.


In [235]:
dataset['test'][0]

{'answers': ['Jamaican Creole English Language', 'Jamaican English'],
 'question': 'what does jamaican people speak?',
 'url': 'http://www.freebase.com/view/en/jamaica'}

In [298]:
import unicodedata
import re

def remove_punc(s):
    res = re.sub(r'[^\w\s]', '', s)
    return res

def normalize(s):
    s = s.replace('\t', ' ')
    s = s.replace('|', '.')
    s = unicodedata.normalize('NFKC', s)
    return s

def normalize_and_remove_punc(s):
    return remove_punc(normalize(s))

In [322]:
id = 529
question = 'predict answer: ' + normalize(dataset['test'][id]['question'])
print(question)
out = greedyPredict(question, model, tokenizer)
true_answers = dataset['test'][id]['answers']
out, true_answers

predict answer: where did paula deen go to school?


('Miami High School', ['Albany High School'])

In [306]:
from tqdm.notebook import tqdm
test_predictions = []
for dp in tqdm(dataset['test']):
    question = 'predict answer: ' + normalize(dp['question'])
    out = greedyPredict(question, model, tokenizer)
    test_predictions.append(out)

In [323]:
num_correct = 0
for i in range(len(test_predictions)):
    pred = remove_punc(test_predictions[i].lower())
    true_ans = [normalize_and_remove_punc(s.lower()) for s in dataset['test'][i]['answers']]
    if pred in true_ans:
        num_correct += 1
num_correct, num_correct/len(test_predictions)

(515, 0.2534448818897638)

In [6]:
input_ids = input_ids.cpu()

In [7]:
# outputs = model.sample(input_ids)
from transformers import (
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    BeamSearchScorer,
)

In [8]:
# fname = 'data/codex-m/test.txt'
fname = 'data/{}/test.txt'.format(dataset_name)
f = open(fname, 'r')
data = []
for line in f:
    data.append(line.strip())
f.close()

In [9]:
len(data)

10642

In [10]:
data[0]

'predict tail: walter harris (artist) | country of citizenship |\tcanada'

In [112]:
def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out

e2wdid = get_entity_wd_id_dict('data/wikidata5m/aliases.txt')

In [14]:
import torch
# data_point = 'predict tail: novalis | occupation |    philosopher'
id = 2
data_point = data[id]
encoder_input_str, target = data_point.split('\t')
encoder_input_str = [encoder_input_str]
encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
num_beams = 2
num_predictions = 2
input_ids = torch.ones((len(encoder_input_str) * num_beams, 1), device=model.device, dtype=torch.long)
input_ids = input_ids * model.config.decoder_start_token_id
model_kwargs = {
    "encoder_outputs": model.get_encoder()(encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True)
}
beam_scorer = BeamSearchScorer(
    batch_size=len(encoder_input_str),
    max_length=model.config.max_length,
    num_beams=num_beams,
    device=model.device,
    num_beam_hyps_to_keep=num_predictions,
    length_penalty=0.3
)
logits_processor = LogitsProcessorList([])

In [189]:
input = 'predict tail: mon oncle benjamin (movie) | original language of film or tv show |'
encoder_input_str = [input]
encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
print(encoder_input_str[0])

predict tail: mon oncle benjamin (movie) | original language of film or tv show |


In [190]:
# outputs = model.beam_search(input_ids, beam_scorer, logits_processor=logits_processor, **model_kwargs)
# print("Beam:", tokenizer.batch_decode(outputs, skip_special_tokens=True))
outputs = model.generate(encoder_input_ids).tolist()[0]
outputs = outputs[1:]
outputs = outputs[:outputs.index(2)]
print(tokenizer.sp.decode(outputs))
# print('Target:', target)

languefrancaise


In [164]:
e2wdid['dante alighieri']

'Q5221150'

In [114]:
wdid2e = {v:k for k, v in e2wdid.items()}

In [182]:
wdid2e['P364']

'original language of film or tv show'

In [31]:
tokenizer.sp.decode(outputs.tolist()[0][1:])

'm/s/doculation/doc/docantc'

In [37]:
outputs.index(2)

6

In [30]:
def getGreedyOutput(model, tokenizer, encoder_input_str):
    encoder_input_str = [encoder_input_str]
    encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids
    outputs = model.generate(encoder_input_ids)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [31]:
def getBeamOutput(model, tokenizer, encoder_input_str, num_beams=10, 
                  num_predictions=3, length_penalty=1.0):
    encoder_input_str = [encoder_input_str]
    encoder_input_ids = tokenizer(encoder_input_str, return_tensors="pt").input_ids.to(model.device)
#     encoder_input_ids = tokenizer(encoder_input_str, padding='max_length', truncation=True, max_length=128, return_tensors="pt").input_ids
    input_ids = torch.ones((len(encoder_input_str) * num_beams, 1), device=model.device, dtype=torch.long)
    input_ids = input_ids * model.config.decoder_start_token_id
    model_kwargs = {
        "encoder_outputs": model.get_encoder()(encoder_input_ids.repeat_interleave(num_beams, dim=0), return_dict=True)
    }
    beam_scorer = BeamSearchScorer(
        batch_size=len(encoder_input_str),
        max_length=model.config.max_length,
        num_beams=num_beams,
        device=model.device,
        num_beam_hyps_to_keep=num_predictions,
        length_penalty=length_penalty
    )
    logits_processor = LogitsProcessorList([])
    outputs = model.beam_search(input_ids, beam_scorer, logits_processor=logits_processor, **model_kwargs)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [32]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [33]:
from tqdm import tqdm
# id = 100
scorer_function = getBeamOutput 
# scorer_function = getGreedyOutput 
num_points = 50
correct = 0
for id in tqdm(range(0, num_points)):
    data_point = data[id]
    input, target = data_point.split('\t')
    predicted = set(scorer_function(model, tokenizer, input))
    print("Predicted:", predicted, "Target:", target)
    if target in predicted:
        correct += 1
print(correct/num_points)

  2%|▏         | 1/50 [00:00<00:16,  2.98it/s]

Predicted: {'france', 'united kingdom of great britain and ireland', 'kingdom of italy'} Target: france


  4%|▍         | 2/50 [00:00<00:14,  3.23it/s]

Predicted: {'jean-baptiste dumas', 'francois-rene de chateaubriand', 'jean-baptiste biot'} Target: gaspard monge
Predicted: {'film actor', 'singer-songwriter', 'film producer'} Target: mandolinist


 10%|█         | 5/50 [00:01<00:10,  4.36it/s]

...

Predicted: {'francois-rene de chateaubriand', 'jean-jacques rousseau', 'euclides da cunha'} Target: augusto righi
Predicted: {'organisation for economic cooperation and development', 'organisation for the prohibition of chemical weapons', 'international telecommunication union'} Target: international bank for reconstruction and development


 96%|█████████▌| 48/50 [00:09<00:00,  5.57it/s]

Predicted: {'senegal', 'egypt', 'sri lanka'} Target: serbia


100%|██████████| 50/50 [00:09<00:00,  5.17it/s]

Predicted: {'sri lanka', 'united kingdom of great britain and ireland', 'united states of america'} Target: mexico
Predicted: {'eugene ionesco', 'euclides da cunha', 'heinrich schliemann'} Target: gloria trevi
0.12


In [34]:
# d = data[0]
# inputs, outputs = d.split('\t')
inputs = 'predict tail: billy idol | languages spoken, written, or signed |'
outputs = 'british english'
x = tokenizer([inputs], return_tensors="pt", add_special_tokens=True)
input_ids = x.input_ids.to(model.device)
attention_mask = x.attention_mask.to(model.device)
labels = tokenizer([outputs], return_tensors="pt").input_ids.to(model.device)
target_tokens = tokenizer(outputs, return_tensors="pt").input_ids[0]

decoder_start_token_id = 0
decoder_input_ids = (
            torch.ones((input_ids.shape[0], 1), dtype=torch.long, device=input_ids.device) * decoder_start_token_id
        )
decoder_input_ids = torch.cat((decoder_input_ids,labels), dim=-1)
decoder_input_ids

tensor([[    0,     3,  2160,    17,  1273, 22269,     1]], device='cuda:0')

In [35]:
x = model(input_ids = input_ids,
         attention_mask=attention_mask,
         labels=labels)

# x = model(input_ids = input_ids,
#          attention_mask=attention_mask,
#          decoder_input_ids = decoder_input_ids)

logits = x.logits[0]
probs = torch.log_softmax(logits, 1)

In [36]:
entities_file = 'data/{}/entity_strings.txt'.format(dataset_name)
with open(entities_file) as f:
    entities_strings = f.read().splitlines()

In [37]:
len(entities_strings)

2034

In [38]:
import numpy as np
def getScore(model, inputs, outputs):
    x = tokenizer([inputs], return_tensors="pt", add_special_tokens=True)
    input_ids = x.input_ids.to(model.device)
    labels = tokenizer([outputs], return_tensors="pt").input_ids.to(model.device)
    target_tokens = labels[0]
    model_output = model(input_ids = input_ids,
             attention_mask=attention_mask,
             labels=labels)
    logits = model_output.logits[0]
    probs = torch.log_softmax(logits, 1)
    score = 0.0
    for i, id in enumerate(target_tokens):
        s = probs[i][id]
        score += s
    return score

def getBatchScore(model, inputs, entities, batch_size=100):
    scores = []
    input_ids = tokenizer(inputs, return_tensors="pt", add_special_tokens=True).input_ids.to(model.device)
    for start_id in range(0, len(entities), batch_size):
        entity_batch = entities[start_id:start_id + batch_size]
        cur_batch_size = len(entity_batch)
        batch_input_ids = input_ids.expand(cur_batch_size, -1)
        labels = tokenizer(entity_batch, 
                           return_tensors="pt",
                           padding=True).input_ids.to(model.device)
        model_output = model(input_ids = batch_input_ids,
                         labels=labels)
        logits = model_output.logits
        probs = torch.log_softmax(logits, -1)
        for i in range(cur_batch_size):
            token_probs = probs[i]
            entity_tokenized = labels[i]
            entity_score = 0.0
            for position, token_id in enumerate(entity_tokenized):
                entity_score += token_probs[position][token_id]
                if token_id == 1:
                    break
            scores.append(entity_score.item())
    return np.array(scores)
        
    


In [39]:
inputs = 'predict tail: billy idol | languages spoken, written, or signed |'
inputs = data[0].split('\t')[0]
scores = getBatchScore(model, inputs, entities_strings)

In [ ]:
all_scores = []
for d in tqdm(data):
    inputs = d.split('\t')[0]
    scores = getBatchScore(model, inputs, entities_strings)
    all_scores.append(scores)

  4%|▍         | 162/3656 [06:55<2:31:33,  2.60s/it]

In [43]:
top = (-scores).argsort()[:10]
entities_strings[top[2]]

'russia'

In [44]:
all_scores_np = np.array(all_scores)

In [45]:
import pickle
fname = 'codex-s_small_1gpu_20000_test_scores.pickle'
pickle.dump(all_scores_np, open(fname, 'wb'))

In [456]:
inputs = 'predict tail: billy idol | languages spoken, written, or signed |'
input_ids = tokenizer(inputs, return_tensors="pt", add_special_tokens=True).input_ids.to(model.device)
input_ids.shape
new_tensor = input_ids.expand(3,-1)
print(input_ids.shape, new_tensor.shape)
new_tensor

torch.Size([1, 16]) torch.Size([3, 16])


tensor([[ 9689,  9891,    10,  2876,    63, 21979,  1820,  8024, 11518,     6,
          1545,     6,    42,  3814,  1820,     1],
        [ 9689,  9891,    10,  2876,    63, 21979,  1820,  8024, 11518,     6,
          1545,     6,    42,  3814,  1820,     1],
        [ 9689,  9891,    10,  2876,    63, 21979,  1820,  8024, 11518,     6,
          1545,     6,    42,  3814,  1820,     1]], device='cuda:0')

In [407]:
inputs = 'predict tail: billy idol | languages spoken, written, or signed |'
outputs = 'english'
getScore(model, inputs, outputs)

tensor(-0.0665, device='cuda:0', grad_fn=<AddBackward0>)

In [423]:
import numpy as np

max_score_entity = ''
max_score = -9999999.
inputs = 'predict tail: billy idol | languages spoken, written, or signed |'
scores = np.array([])
for e in tqdm(entities_strings):
    outputs = e
    s = getScore(model, inputs, outputs)
    scores = np.append(scores, s.detach().cpu())


100%|██████████| 17050/17050 [06:38<00:00, 42.79it/s]


In [27]:
top = (-scores).argsort()[:10]

In [29]:
entities_strings[top[0]]

'english'

In [403]:
score = 0.0
for i, id in enumerate(target_tokens):
    s = probs[i][id]
    score += s
# score = torch.max(logits[0])
print(score.item())
# average_score_per_token = score.item()/(len(target_tokens))
# print(average_score_per_token, len(target_tokens))
# score = score.item()
# print(score/len(target_tokens), score)

-6.565360069274902


In [326]:
print(logits[0][:10])
print(torch.max(logits[0]))

tensor([  1.8634,   4.0730,  -2.0617,  11.3171, -12.6902,  -0.2787,  -0.1332,
          1.3992,   4.9949,   1.2909], device='cuda:0',
       grad_fn=<SliceBackward>)
tensor(15.8887, device='cuda:0', grad_fn=<MaxBackward1>)


In [325]:
tokenizer.convert_ids_to_tokens([torch.argmax(logits[0])])

['▁english']

In [4]:
from dataset import T5_Dataset

In [5]:
valid_dataset = T5_Dataset('test', dataset_name='codex-m')

100%|██████████| 20622/20622 [00:00<00:00, 840393.08it/s]


In [6]:
from eval_accelerate import removePadding, eval

In [7]:
class Args:
    batch_size = 200
args=Args()

In [8]:
acc = eval(model, valid_dataset, args)

100%|██████████| 104/104 [00:59<00:00,  1.74batches/s]


In [9]:
acc

0.10876733585491223

In [8]:
actual = tokenizer("international development association", return_tensors="pt").input_ids[0].numpy()

In [9]:
actual

array([1038,  606, 6028,    1])

In [10]:
predicted = outputs[0][1:].numpy()

In [11]:
predicted

array([ 1038,  2137,    21, 20532,    11,   606,     1])

In [30]:
actual == predicted

array([ True,  True,  True,  True])

In [25]:
actual.numpy()

array([1038,  606, 6028,    1])